# Projet évaluation de performance
## génération automatique d'un emploi du temps
#### _Hoel Roquinarc'h - Théo Manéa_



### Préprocessing
L'algorithme utilise 2 bases de données sous format texte présentées ainsi :
- subject.txt -> id du sujet/nom de la matière/n° de la classe/salle du cours/id du professeur
- lecturers.txt -> id du professeur/ nom du professeur 
<br/><br/> 
Cette première partie de l'algorithme a pour objectif de récupérer ces 2 bases de données afin d'en extraire les données interessantes.


In [2]:
import numpy as np
import pandas as pd

In [3]:
data=open("data/subject.txt")
list_sujet_r=data.readlines()

list_sujet = []
for d in list_sujet_r:
    list_sujet.append(d.replace(" ", ""))

data=open("data/lecturers.txt")
list_prof_r=data.readlines()

list_prof = []
for d in list_prof_r:
    list_prof.append(d.replace(" ", ""))


nbTsPerDay = 5

In [4]:
#list prof -> dataframe
data_lines=[]
for d in list_prof : 
    data_lines.append(d[:len(d)-1].split("\t"))
    
list_prof=pd.DataFrame(data_lines,columns=["id","nom"])

In [5]:
#list sujet -> dataframe
data_lines=[]
for d in list_sujet : 
    data_lines.append(d[:len(d)-1].split("\t"))
    
list_sujet=pd.DataFrame(data_lines,columns=["id","matiere","class","room","id_prof"])

In [6]:
'''list_sujet.insert(0,list_sujet["prof"],"")
for elt in range (len(list_sujet)):
    list_sujet["prof"][elt]=list_prof["nom"][int(list_sujet["id_prof"][elt])]'''

'list_sujet.insert(0,list_sujet["prof"],"")\nfor elt in range (len(list_sujet)):\n    list_sujet["prof"][elt]=list_prof["nom"][int(list_sujet["id_prof"][elt])]'

In [7]:
list_sujet


,id,matiere,class,room,id_prof
0,000,Francais,001,R1,001
1,001,Francais,001,R1,001
2,002,Francais,001,R1,001
3,003,Francais,001,R1,001
4,004,Mathematique,001,R2,002
...,...,...,...,...,...
87,087,Physique-chimie,004,R17,017
88,088,Physique-chimie,004,R7,017
89,089,Thechnologie,004,R18,018
90,090,Sport,004,R10,009


### Création des vertex
Notre algorithme crée un graphe a partir des données récoltées.
<br/> les sommets représentent les différents sujets et les arettes représentes les contraintes entre les différents sujets.
<br/> Les arettes sont tracées sous les conditions suivantes :
- si les deux sujets sont destinés a la même classe
- si les deux sujets sont enseignés dns la même salle
- si les deux sujets sont enseignés par le même intervenant

In [8]:
%%time
vertex_tab = np.zeros((len(list_sujet),len(list_sujet)),dtype=int)
for i in range(len(vertex_tab)):
    for j in range (len(vertex_tab[i])):
        if (i == j):
            vertex_tab[i][j] = int(-1)
        elif (list_sujet["class"][i] == list_sujet["class"][j]):
            vertex_tab[i][j] = int(1)
        elif (list_sujet["room"][i] == list_sujet["room"][j]):
            vertex_tab[i][j] = int(1)
        elif (list_sujet["id_prof"][i] == list_sujet["id_prof"][j]):
            vertex_tab[i][j] = int(1)

CPU times: total: 62.5 ms
Wall time: 108 ms


In [9]:
vertex_df = pd.DataFrame(vertex_tab)

In [10]:
vertex_df

,0,1,2,3,4,5,6,7,8,9,...,82,83,84,85,86,87,88,89,90,91
0,-1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,-1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,-1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,-1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,-1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,-1,1,1,1,1
88,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,-1,1,1,1
89,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,-1,1,1
90,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,-1,1


### Welsh-Powell
Dans cette partie nous utilisons l'algorithme Welsh-Powell afin de fait un coloration du graphe précédement construit.
Ainsi, les couleurs représentent les périodes de temps et ainsi 2 sujets qui ont des arretes les reliants(donc des contraintes l'un par rapport à l'autre) ne pourrons pas se déroulé au même moment

In [11]:
%%time
def Welsh_powell(tab):
    degree = {}
    color = {}
    colored = []
    clr = 0
    for i in range(len(tab)):
        colored.append(False)
        degree[i] = 0
        for j in range(len(tab)):
            if (tab[i][j] != -1):
                degree[i] += tab[i][j]
    degree = dict(sorted(degree.items(), key=lambda x:x[1], reverse=True))
    sorted_list = list(degree.keys())
    
    while False in colored:
        selected = sorted_list[0]
        colored[selected] = True
        color[selected] = clr
        sorted_list.remove(selected)
        slist = [selected]
        for sommet in sorted_list:
            flag = True
            for x in slist:
                if (tab[x][sommet] == 1):
                    flag = False
            if flag:
                colored[sommet] = True
                color[sommet] = clr
                sorted_list.remove(sommet)
                slist.append(sommet)
           
        clr += 1       
    
    print(str(clr) + " periodes de cours sont néssescaires")
    color = dict(sorted(color.items(), key=lambda x:x[0]))        
    return color

a = Welsh_powell(vertex_tab)
a

24 periodes de cours sont néssescaires
CPU times: total: 0 ns
Wall time: 4 ms


{0: 4,
 1: 5,
 2: 6,
 3: 7,
 4: 8,
 5: 9,
 6: 10,
 7: 11,
 8: 12,
 9: 13,
 10: 14,
 11: 16,
 12: 15,
 13: 17,
 14: 18,
 15: 20,
 16: 19,
 17: 2,
 18: 21,
 19: 3,
 20: 23,
 21: 0,
 22: 1,
 23: 8,
 24: 9,
 25: 10,
 26: 12,
 27: 4,
 28: 5,
 29: 11,
 30: 13,
 31: 6,
 32: 7,
 33: 14,
 34: 18,
 35: 19,
 36: 20,
 37: 15,
 38: 16,
 39: 17,
 40: 0,
 41: 22,
 42: 1,
 43: 21,
 44: 2,
 45: 3,
 46: 0,
 47: 1,
 48: 2,
 49: 3,
 50: 12,
 51: 13,
 52: 14,
 53: 15,
 54: 8,
 55: 9,
 56: 10,
 57: 11,
 58: 16,
 59: 17,
 60: 18,
 61: 19,
 62: 20,
 63: 6,
 64: 21,
 65: 7,
 66: 22,
 67: 4,
 68: 5,
 69: 12,
 70: 13,
 71: 14,
 72: 15,
 73: 6,
 74: 7,
 75: 8,
 76: 9,
 77: 0,
 78: 1,
 79: 2,
 80: 3,
 81: 18,
 82: 19,
 83: 16,
 84: 17,
 85: 22,
 86: 4,
 87: 20,
 88: 5,
 89: 21,
 90: 10,
 91: 11}

In [13]:
def create_edt(tab,df,cls):
    nbLines = nbTsPerDay
    nbCol = 5
    edt = []
    for _ in range(nbLines+1):
        ligne = [' '] * nbCol
        edt.append(ligne)
    #edt = np.zeros((nbTsPerDay,5),dtype=)
    for i in range(len(tab)):
        if (df["class"][i] == cls):
            idc = int(df["id"][i])
            for key,value in tab.items():
                if key == idc:
                    order = value
            cpt = 0
            while(order >= nbLines):
                cpt += 1
                order -= nbLines
                
            if(order+0.5 >= nbLines/2):
                order += 1
            edt[order][cpt] = df["matiere"][idc]
            
    return edt
            
    
    
x = create_edt(a,list_sujet,"002")
edt = pd.DataFrame(x,columns=["lundi","mardi","mercredi","jeudi","vendredi"])
edt

,lundi,mardi,mercredi,jeudi,vendredi
0,SVT,Mathematique,Francais,LV2,LV1
1,Physique-chimie,Histoire-geographie,Mathematique,LV2,Thechnologie
2,,,,,
3,Sport,Histoire-geographie,Francais,SVT,Physique-chimie
4,Sport,Francais,Mathematique,LV1,
5,Mathematique,Francais,Histoire-geographie,LV1,
